In [1]:
import tensorflow as tf

In [2]:
def _get_weight(name, shape, dtype=tf.float32, relu=True):
    if relu:
        return tf.get_variable(name=name, shape=shape, dtype=dtype, 
                                initializer=tf.variance_scaling_initializer(scale=2.))
    else:
        return tf.get_variable(name=name, shape=shape, dtype=dtype,
                                initializer=tf.variance_scaling_initializer(distribution='uniform'))


def _get_bias(name, shape, dtype=tf.float32):
    return tf.get_variable(name=name, shape=shape, dtype=dtype,
                            initializer=tf.zeros_initializer())

In [3]:
def _conv_layer(inputs, dropout_keep_prob, n_convs, training):
    
    def _inception(name, inputs, in_channels, out_channels, training):
        
        def batch_norm(x):
            mean, var = tf.nn.moments(x, axes=[0, 1, 2])
            bn = tf.nn.batch_normalization(x, mean, var, 0, 1, 1e-3, name='batch_normal')
            return bn

        def _conv(name, inputs, in_channels, out_channels, ksize, training):
            with tf.variable_scope(name):
                if ksize == 1:
                    kernel = _get_weight(name='kernel', shape=[ksize, ksize, in_channels, out_channels], relu=False)
                    bias = _get_bias(name='bias', shape=[out_channels])
                    conv = tf.nn.conv2d(inputs, kernel, [1, 1, 1, 1], padding='SAME')
                    conv_add_bias = tf.nn.bias_add(conv, bias)
                    #bn = tf.layers.batch_normalization(conv_add_bias, name='batch_normal')
                    #bn = batch_norm(conv_add_bias)
                    bn = conv_add_bias
                    #bn = tf.layers.batch_normalization(conv_add_bias, training=training, name='batch_normal')            
                    relu = tf.nn.relu(bn, name='relu')
                else:
                    kernel = _get_weight(name='kernel1', shape=[1, ksize, in_channels, in_channels], relu=False)
                    conv = tf.nn.conv2d(inputs, kernel, [1, 1, 1, 1], padding='SAME')
                    relu = tf.nn.relu(conv, name='relu1')
                    #relu = conv
                    
                    kernel = _get_weight(name='kernel2', shape=[ksize, 1, in_channels, out_channels], relu=False)
                    conv = tf.nn.conv2d(conv, kernel, [1, 1, 1, 1], padding='SAME')
                    relu = tf.nn.relu(conv, name='relu2')
            return relu
       
        with tf.variable_scope(name):
            k1_channels = out_channels / 2
            conv1_1 = _conv('conv1_1', inputs, in_channels, k1_channels, ksize=1, training=training)
            conv1_2 = _conv('conv1_2', conv1_1, k1_channels, k1_channels, ksize=3, training=training)
            pool1 = tf.nn.max_pool(conv1_2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='max_pool1')

            k1_channels = out_channels / 8 * 3
            conv2_1 = _conv('conv2_1', inputs, in_channels, k1_channels, ksize=1, training=training)
            conv2_2 = _conv('conv2_2', conv2_1, k1_channels, k1_channels, ksize=3, training=training)
            conv2_3 = _conv('conv2_3', conv2_2, k1_channels, k1_channels, ksize=3, training=training)
            pool2 = tf.nn.max_pool(conv2_3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='max_pool2')

            pool3 = tf.nn.max_pool(inputs, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='max_pool2')
            conv3 = _conv('conv3', pool3, in_channels, out_channels/8, ksize=1, training=training)

            concated = tf.concat([pool1, pool2, conv3], axis=3)
            dropout = tf.nn.dropout(concated, dropout_keep_prob, name='dropout')
        
        return dropout

    
    for i in range(n_convs):
        if i == 0:
            conv = _inception('inception1', inputs, 3, 8, training)
        else:
            conv = _inception('inception%d' % (i+1), conv, 2**(i+2), 2**(i+3), training)
    
    return conv

In [4]:
def _rnn_layer(inputs, dropout_keep_prob):
    with tf.variable_scope('bidirectional_rnn'):
        col_wise = tf.transpose(inputs, [0, 2, 1, 3])
        shape = col_wise.get_shape().as_list()
        col_size = shape[2] * shape[3]
        rnn_inputs = tf.reshape(col_wise, [-1, shape[1], col_size])
        
        num_units = int(col_size)
        cell_unit = tf.nn.rnn_cell.GRUCell
        cell_fw = cell_unit(num_units, name='cell_fw')
        cell_fw_dropout = tf.nn.rnn_cell.DropoutWrapper(cell_fw, output_keep_prob=dropout_keep_prob)        
        cell_bw = cell_unit(num_units, name='cell_bw')
        cell_bw_dropout = tf.nn.rnn_cell.DropoutWrapper(cell_bw, output_keep_prob=dropout_keep_prob)
        
        outputs_tuple, final_state_tuple = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, rnn_inputs, dtype=tf.float32)
        outputs = tf.concat(axis=2, values=outputs_tuple)
    
        return outputs[:, -1], final_state_tuple, (cell_fw, cell_bw)

In [5]:
def _outputs_layer(inputs, dropout_keep_prob, n_len, n_classes, n_denses):    
    
    def _dense(name, inputs, in_dim, out_dim):
        with tf.variable_scope(name):
            w = _get_weight(name='weight', shape=[in_dim, out_dim])
            b = _get_bias(name='bias', shape=[out_dim])
            dense = tf.matmul(inputs, w) + b
            bn = tf.layers.batch_normalization(dense, name='batch_normal')
            relu = tf.nn.relu(bn)
        return relu
    
    with tf.variable_scope('output'):
        dim = inputs.get_shape().as_list()[-1]
        dense = inputs
        for i in range(n_denses):
            dense = _dense('dense%d' % (i+1), dense, dim, dim/2)
            dim /= 2
        dropout = tf.nn.dropout(dense, dropout_keep_prob, name='dropout')
    
        w = _get_weight(name='weight', shape=[dim, n_classes*n_len])
        b = _get_bias(name='bias', shape=[n_classes*n_len])
        final = tf.matmul(dropout, w) + b
        logits = tf.reshape(final, [-1, n_len, n_classes])
    
    return logits

In [6]:
def _eval(logits, labels, training):
    class _Eval(object):
        pass
    result = _Eval()

    name = 'train_eval' if training else 'test_eval'
    with tf.variable_scope(name):
        losses = []
        n_len = logits.get_shape().as_list()[1]
        for i in range(n_len):
            losses.append(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits[:, i], labels=labels[:,i]))
        result.loss = tf.reduce_mean(losses, name='mean_loss')
        tf.summary.scalar('loss', result.loss)

        if not training:
            preds = tf.argmax(logits, 2, name='preds')
            result.preds = preds

            correct_preds = tf.reduce_all(tf.equal(preds, labels), 1)
            result.acc = tf.reduce_mean(tf.cast(correct_preds, tf.float32), name='acc')
            result.acc_ = tf.reduce_mean(tf.cast(tf.equal(preds, labels), tf.float32), name='acc_')
            tf.summary.scalar('accuracy', result.acc)
            tf.summary.scalar('accuracy_', result.acc_)

    return result

In [7]:
class Model(object):
    
    def __init__(self, images, labels, n_len, n_classes, n_convs=4, n_denses=1, 
                 learning_rate=1 / 1024, batch_size=64, training=True):        
        if training:
            self.batch_size = batch_size
            dropout_keep_prob = 4 / 5
        else:
            self.batch_size = 32
            dropout_keep_prob = 1.
        
        conv = _conv_layer(inputs=images, dropout_keep_prob=dropout_keep_prob, n_convs=n_convs, training=training)
        print('_conv_layer\t', conv)
        
        rnn_outputs, self.final_state, self.cells = _rnn_layer(inputs=conv, 
                                                               dropout_keep_prob=dropout_keep_prob)
        self.init_state = (self.cells[0].zero_state(self.batch_size, dtype=tf.float32),
                           self.cells[1].zero_state(self.batch_size, dtype=tf.float32))
        print('_rnn_layer\t', rnn_outputs)

        logits = _outputs_layer(inputs=rnn_outputs, dropout_keep_prob=dropout_keep_prob, 
                                n_len=n_len, n_classes=n_classes, n_denses=n_denses)
        print('_output_layer\t', logits, '\n')
        
        self.eval = _eval(logits=logits, labels=labels, training=training)
        
        
        if training:
            opt = tf.train.AdamOptimizer(learning_rate)
            #update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            #with tf.control_dependencies(update_ops):
            gradients, _ =  tf.clip_by_global_norm(tf.gradients(self.eval.loss, tf.trainable_variables()), 5)
            self.train_op = opt.apply_gradients(zip(gradients, tf.trainable_variables()))